In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams
from sentence_transformers import SentenceTransformer
import numpy as np
from pathlib import Path
import json
import pandas as pd
import torch
#If you already saved the model locally and are using docker
client = QdrantClient(host="localhost", port=6333, prefer_grpc=True)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer("models/multilingual-e5-large", device=device)

# Run docker container before
# docker run -d --name qdrant -p 6333:6333 -p 6334:6334 -v /home/Loris/EPFL/MA3/ML/project2/project2Rag/qdrant_storage:/qdrant/storage qdrant/qdrant:latest

# VERY IMPORTANT : to query the remote qdrand docker container over ssh, run the following in a terminale before:
# ssh -L 6333:localhost:6333 -L 6334:localhost:6334 <your username>@dclgpusrv.epfl.ch -N
#Make sure to have your ssh keys before
#This forwards the qdrant ports to your local machine


In [2]:
from qdrant_client.http import models as rest
#get articles count from Qdrant DB 

collection_name = "wikipedia_fr"
total = client.count(collection_name=collection_name, exact=True).count
print(f"Total articles stored: {total}")

Total articles stored: 2556402


In [3]:
import polars as pl
import numpy as np
from tqdm import tqdm

# Keep as LazyFrame - don't collect the full dataset!
df_lazy = pl.scan_parquet("articles_fr_merged.parquet").filter(
    pl.col('link_count') > 0
)

# Only collect a small sample to verify structure
sample = df_lazy.head(3).collect()

print(f"📚 Dataset loaded as LazyFrame (not in memory)")
print(f"   Schema: {df_lazy.collect_schema()}")
print(f"\n   Link structure example:")
print(f"   {sample.select('links').to_series()[0][:2]}")

📚 Dataset loaded as LazyFrame (not in memory)
   Schema: Schema([('id', Int64), ('title', String), ('text', String), ('links', List(Struct({'full_url': String, 'start_idx': Int64, 'anchor': String, 'href_raw': String, 'href_decoded': String}))), ('link_count', UInt32), ('text_withoutHref', String)])

   Link structure example:
   shape: (2,)
Series: '' [struct[5]]
[
	{"https://fr.wikipedia.org/wiki/Moulins%20%28Allier%29",25,"Moulins","Moulins%20%28Allier%29","Moulins (Allier)"}
	{"https://fr.wikipedia.org/wiki/Allier%20%28d%C3%A9partement%29",83,"Allier","Allier%20%28d%C3%A9partement%29","Allier (département)"}
]


In [4]:
import torch
import re
from typing import List, Dict, Tuple
from urllib.parse import unquote

def create_url_to_id_mapping_from_qdrant(client, collection_name: str = "wikipedia_fr") -> Dict[str, int]:
    """Build URL → ID mapping using articles in Qdrant."""
    from urllib.parse import quote
    
    url_to_id = {}
    id_to_title = {}
    
    print(f"🗺️  Building URL → ID mapping from '{collection_name}'...")
    
    offset = None
    batch_size = 1000
    total_processed = 0
    
    while True:
        points, offset = client.scroll(
            collection_name=collection_name,
            limit=batch_size,
            offset=offset,
            with_payload=True,
            with_vectors=False
        )
        
        if not points:
            break
        
        for point in points:
            article_id = point.payload.get("id")
            title = point.payload.get("title", "")
            
            if not article_id or not title:
                continue
            
            id_to_title[article_id] = title
            
            # Create URL pattern variations
            patterns = [
                title,
                title.replace(" ", "_"),
                quote(title.replace(" ", "_"), safe=""),
                title.lower(),
                title.lower().replace(" ", "_"),
                title.lower().replace(" ", "%20"),
            ]
            
            for pattern in patterns:
                url_to_id[pattern] = article_id
        
        total_processed += len(points)
        
        if offset is None:
            break
    
    print(f"✅ Created {len(url_to_id):,} URL mappings from {total_processed:,} articles")
    return url_to_id, id_to_title

# Build mappings
url_to_id, id_to_title = create_url_to_id_mapping_from_qdrant(client, "wikipedia_fr")

🗺️  Building URL → ID mapping from 'wikipedia_fr'...


✅ Created 12,682,763 URL mappings from 2,556,402 articles


In [5]:
def extract_sentences_with_links_from_positions(
    text: str, 
    links: list
) -> List[Dict]:
    """Extract sentences containing hyperlinks using position information."""
    if not text or not links or len(links) == 0:
        return []
    
    sorted_links = sorted(links, key=lambda x: x.get("start_idx", 0))
    sentence_pattern = r'[.!?]+\s+'
    sentences = re.split(sentence_pattern, text)
    
    results = []
    current_pos = 0
    
    for sentence in sentences:
        if not sentence.strip():
            continue
        
        sent_start = text.find(sentence, current_pos)
        if sent_start == -1:
            continue
        sent_end = sent_start + len(sentence)
        
        links_in_sent = []
        for link in sorted_links:
            link_pos = link.get("start_idx", -1)
            
            if sent_start <= link_pos < sent_end:
                links_in_sent.append({
                    'anchor': link.get('anchor', ''),
                    'href_decoded': link.get('href_decoded', ''),
                    'position': link_pos,
                })
        
        if links_in_sent:
            results.append({
                'sentence': sentence.strip(),
                'links_in_sentence': links_in_sent,
                'start_pos': sent_start,
                'num_links': len(links_in_sent)
            })
        
        current_pos = sent_end
    
    return results

print("✅ Sentence extraction function defined")

✅ Sentence extraction function defined


In [6]:
def create_test_dataset_optimized(
    df_lazy: pl.LazyFrame,
    url_to_id: Dict[str, int],
    max_sentences: int = 100,
    min_links_per_sentence: int = 1,
    sample_articles: int = 5000  # Sample size to avoid loading all 2.5M
) -> List[Dict]:
    """
    Create test dataset using LazyFrame streaming.
    Only loads a sample of articles into memory.
    """
    print(f"🔍 Creating test dataset (max {max_sentences} sentences)...")
    print(f"   Sampling {sample_articles:,} articles from LazyFrame...")
    
    # Sample articles using LazyFrame operations
    # This only materializes the sample, not the full dataset
    sampled_df = (
        df_lazy
        .filter(pl.col('link_count') > 0)
        .filter(pl.col('text_withoutHref').is_not_null())
        .filter(pl.col('text_withoutHref').str.len_chars() > 100)  # Skip very short articles
        .select(['id', 'title', 'text_withoutHref', 'links'])  # Only needed columns
        .head(sample_articles * 3)  # Get more than needed to allow for filtering
        .collect(streaming=True)  # Stream processing
        .sample(n=min(sample_articles, sample_articles * 3), shuffle=True, seed=42)
    )
    
    print(f"   Loaded {len(sampled_df):,} articles into memory")
    
    test_data = []
    articles_processed = 0
    
    for row in tqdm(sampled_df.iter_rows(named=True), total=len(sampled_df), desc="Processing"):
        if len(test_data) >= max_sentences:
            break
        
        article_id = row["id"]
        article_title = row["title"]
        text = row.get("text_withoutHref", "")
        links_raw = row.get("links", [])
        
        if links_raw is None:
            continue
        links = list(links_raw) if hasattr(links_raw, '__iter__') else []
        
        if not text or len(links) == 0:
            continue
        
        # Extract sentences with links
        sentences_with_links = extract_sentences_with_links_from_positions(text, links)
        
        for sent_data in sentences_with_links:
            if len(test_data) >= max_sentences:
                break
            
            sentence = sent_data["sentence"]
            links_in_sent = sent_data["links_in_sentence"]
            
            # Map links to article IDs
            ground_truth_links = []
            for link in links_in_sent:
                href_decoded = link["href_decoded"]
                anchor = link["anchor"]
                
                if not anchor or not anchor.strip():
                    continue
                
                target_id = url_to_id.get(href_decoded)
                if not target_id:
                    for variation in [
                        href_decoded.replace("_", " "),
                        href_decoded.replace("%20", " "),
                        href_decoded.lower(),
                        href_decoded.lower().replace("_", " ")
                    ]:
                        target_id = url_to_id.get(variation)
                        if target_id:
                            break
                
                if target_id and target_id != article_id:
                    ground_truth_links.append({
                        'anchor': anchor,
                        'target_id': target_id,
                        'href_decoded': href_decoded
                    })
            
            if len(ground_truth_links) >= min_links_per_sentence:
                test_data.append({
                    'source_article_id': article_id,
                    'source_article_title': article_title,
                    'sentence': sentence,
                    'ground_truth_links': ground_truth_links,
                    'num_ground_truth': len(ground_truth_links)
                })
        
        articles_processed += 1
    
    # Free memory
    del sampled_df
    import gc
    gc.collect()
    
    print(f"\n✅ Created test dataset:")
    print(f"   Sentences: {len(test_data)}")
    print(f"   Articles processed: {articles_processed}")
    total_gt_links = sum(t['num_ground_truth'] for t in test_data)
    print(f"   Total ground truth links: {total_gt_links}")
    
    return test_data

# Create test dataset - pass LazyFrame, not collected DataFrame
test_data = create_test_dataset_optimized(
    df_lazy,  # LazyFrame, not df
    url_to_id, 
    max_sentences=200,
    min_links_per_sentence=1,
    sample_articles=5000  # Only load 5K articles into memory
)

🔍 Creating test dataset (max 200 sentences)...
   Sampling 5,000 articles from LazyFrame...


/tmp/ipykernel_24626/799159996.py:24: DeprecationWarning: the `streaming` parameter was deprecated in 1.25.0; use `engine` instead.
  .collect(streaming=True)  # Stream processing


   Loaded 5,000 articles into memory


Processing:   0%|          | 9/5000 [00:00<03:52, 21.50it/s]



✅ Created test dataset:
   Sentences: 200
   Articles processed: 9
   Total ground truth links: 285


In [7]:
def predict_links_for_sentences(
    client,
    model,
    test_data: List[Dict],
    collection_name: str = "wikipedia_fr",
    top_k: int = 10,
    min_similarity: float = 0.5
) -> List[Dict]:
    """
    Predict links by finding similar articles in Qdrant.
    
    Approach: Encode sentence → Find similar articles → Suggest those articles as links
    """
    print(f"🔮 Predicting links for {len(test_data)} sentences...")
    
    # Extract all sentences
    sentences = [t['sentence'] for t in test_data]
    source_ids = [t['source_article_id'] for t in test_data]
    
    # Encode all sentences
    print("   Encoding sentences...")
    embeddings = model.encode(
        sentences,
        normalize_embeddings=True,
        show_progress_bar=True,
        convert_to_numpy=True,
        device=device
    )
    
    predictions = []
    
    print("   Querying Qdrant...")
    for i, (sentence, embedding, source_id) in enumerate(tqdm(
        zip(sentences, embeddings, source_ids), 
        total=len(sentences)
    )):
        # Search for similar articles
        search_results = client.search(
            collection_name=collection_name,
            query_vector=embedding.tolist(),
            limit=top_k + 1,  # Extra to account for self-match
            score_threshold=min_similarity
        )
        
        # Filter out source article and collect predictions
        predicted_articles = []
        for result in search_results:
            article_id = result.payload.get('id')
            
            # Skip self-match
            if article_id == source_id:
                continue
            
            predicted_articles.append({
                'article_id': article_id,
                'article_title': result.payload.get('title', f'ID {article_id}'),
                'similarity_score': result.score,
                'text_preview': result.payload.get('text_withoutHref', '')[:150] + '...'
            })
        
        # Limit to top_k after filtering
        predicted_articles = predicted_articles[:top_k]
        
        predictions.append({
            **test_data[i],
            'predicted_articles': predicted_articles,
            'num_predictions': len(predicted_articles)
        })
    
    print(f"✅ Generated predictions for {len(predictions)} sentences")
    return predictions

# Generate predictions
predictions = predict_links_for_sentences(
    client,
    model,
    test_data,
    collection_name="wikipedia_fr",
    top_k=10,
    min_similarity=0.5
)

🔮 Predicting links for 200 sentences...
   Encoding sentences...


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

   Querying Qdrant...


  0%|          | 0/200 [00:00<?, ?it/s]/tmp/ipykernel_24626/2008121553.py:38: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = client.search(
100%|██████████| 200/200 [00:04<00:00, 45.66it/s]

✅ Generated predictions for 200 sentences


In [8]:
def evaluate_predictions(predictions: List[Dict]) -> Dict:
    """
    Evaluate predictions against ground truth.
    
    A prediction is correct if the predicted article ID matches
    any of the ground truth target article IDs.
    """
    total_ground_truth = 0
    total_predictions = 0
    correct_predictions = 0
    
    results_per_sentence = []
    
    for pred in predictions:
        gt_ids = set(link['target_id'] for link in pred['ground_truth_links'])
        pred_ids = set(p['article_id'] for p in pred['predicted_articles'])
        
        # Count matches
        matches = gt_ids & pred_ids
        
        total_ground_truth += len(gt_ids)
        total_predictions += len(pred_ids)
        correct_predictions += len(matches)
        
        # Per-sentence metrics
        precision = len(matches) / len(pred_ids) if pred_ids else 0
        recall = len(matches) / len(gt_ids) if gt_ids else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        
        results_per_sentence.append({
            **pred,
            'matched_ids': list(matches),
            'precision': precision,
            'recall': recall,
            'f1': f1
        })
    
    # Overall metrics
    overall_precision = correct_predictions / total_predictions if total_predictions > 0 else 0
    overall_recall = correct_predictions / total_ground_truth if total_ground_truth > 0 else 0
    overall_f1 = 2 * overall_precision * overall_recall / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0
    
    print("\n" + "="*60)
    print("📊 EVALUATION RESULTS")
    print("="*60)
    print(f"\n📈 Overall Metrics:")
    print(f"   Precision: {overall_precision:.3f} ({correct_predictions}/{total_predictions})")
    print(f"   Recall:    {overall_recall:.3f} ({correct_predictions}/{total_ground_truth})")
    print(f"   F1 Score:  {overall_f1:.3f}")
    print(f"\n📋 Dataset Stats:")
    print(f"   Test sentences: {len(predictions)}")
    print(f"   Total ground truth links: {total_ground_truth}")
    print(f"   Total predictions: {total_predictions}")
    print(f"   Correct predictions: {correct_predictions}")
    
    return {
        'precision': overall_precision,
        'recall': overall_recall,
        'f1': overall_f1,
        'total_ground_truth': total_ground_truth,
        'total_predictions': total_predictions,
        'correct_predictions': correct_predictions,
        'results_per_sentence': results_per_sentence
    }

# Evaluate
eval_results = evaluate_predictions(predictions)


📊 EVALUATION RESULTS

📈 Overall Metrics:
   Precision: 0.002 (3/2000)
   Recall:    0.011 (3/285)
   F1 Score:  0.003

📋 Dataset Stats:
   Test sentences: 200
   Total ground truth links: 285
   Total predictions: 2000
   Correct predictions: 3


In [9]:
def display_detailed_analysis(eval_results: Dict, num_samples: int = 5):
    """Show detailed examples of predictions vs ground truth."""
    
    results = eval_results['results_per_sentence']
    
    # Sort by F1 score to show best and worst
    sorted_results = sorted(results, key=lambda x: x['f1'], reverse=True)
    
    print("\n" + "="*80)
    print("🎯 BEST PREDICTIONS (Highest F1)")
    print("="*80)
    
    for i, result in enumerate(sorted_results[:num_samples]):
        if result['f1'] == 0:
            continue
        print(f"\n{'─'*80}")
        print(f"Example {i+1}: Article '{result['source_article_title']}'")
        print(f"Sentence: '{result['sentence'][:120]}...'")
        print(f"Precision: {result['precision']:.2f} | Recall: {result['recall']:.2f} | F1: {result['f1']:.2f}")
        
        print(f"\n✅ Ground Truth ({result['num_ground_truth']}):")
        for gt in result['ground_truth_links']:
            target_title = id_to_title.get(gt['target_id'], f"ID {gt['target_id']}")
            matched = "✓" if gt['target_id'] in result['matched_ids'] else " "
            print(f"   [{matched}] '{gt['anchor']}' → {target_title}")
        
        print(f"\n🔮 Top Predictions ({min(5, result['num_predictions'])}):")
        for pred in result['predicted_articles'][:5]:
            matched = "✓" if pred['article_id'] in result['matched_ids'] else "✗"
            print(f"   [{matched}] {pred['article_title']} (sim: {pred['similarity_score']:.3f})")
    
    print("\n" + "="*80)
    print("❌ WORST PREDICTIONS (Lowest F1, with predictions)")
    print("="*80)
    
    # Get worst predictions that have at least some predictions
    worst = [r for r in sorted_results if r['num_predictions'] > 0][-num_samples:]
    
    for i, result in enumerate(worst):
        print(f"\n{'─'*80}")
        print(f"Example {i+1}: Article '{result['source_article_title']}'")
        print(f"Sentence: '{result['sentence'][:120]}...'")
        print(f"Precision: {result['precision']:.2f} | Recall: {result['recall']:.2f} | F1: {result['f1']:.2f}")
        
        print(f"\n✅ Ground Truth ({result['num_ground_truth']}):")
        for gt in result['ground_truth_links']:
            target_title = id_to_title.get(gt['target_id'], f"ID {gt['target_id']}")
            print(f"   '{gt['anchor']}' → {target_title}")
        
        print(f"\n🔮 Top Predictions ({min(3, result['num_predictions'])}):")
        for pred in result['predicted_articles'][:3]:
            print(f"   ✗ {pred['article_title']} (sim: {pred['similarity_score']:.3f})")

# Display analysis
display_detailed_analysis(eval_results, num_samples=5)


🎯 BEST PREDICTIONS (Highest F1)

────────────────────────────────────────────────────────────────────────────────
Example 1: Article 'Haute qualité environnementale'
Sentence: 'La démarche HQE étant critiquée pour son manque de lisibilité et sa défense des intérêts commerciaux des industriels : l...'
Precision: 0.10 | Recall: 1.00 | F1: 0.18

✅ Ground Truth (1):
   [✓] 'haute performance énergétique' → Haute performance énergétique

🔮 Top Predictions (5):
   [✗] Association HQE (sim: 0.860)
   [✗] Association des professionnels de la construction et de l'habitation du Québec (sim: 0.838)
   [✗] Qualité environnementale (sim: 0.831)
   [✗] Association des constructeurs européens d'automobiles (sim: 0.830)
   [✗] The Society of British Interior Design (sim: 0.829)

────────────────────────────────────────────────────────────────────────────────
Example 2: Article 'Haute qualité environnementale'
Sentence: 'La HQE serait un « impensé politique » permettant de générer du profit sur le dos

In [10]:
def print_summary_statistics(eval_results: Dict):
    """Print summary statistics about the evaluation."""
    
    results = eval_results['results_per_sentence']
    
    # Count sentences with at least one correct prediction
    sentences_with_match = sum(1 for r in results if len(r['matched_ids']) > 0)
    
    # Average metrics
    avg_precision = np.mean([r['precision'] for r in results])
    avg_recall = np.mean([r['recall'] for r in results])
    avg_f1 = np.mean([r['f1'] for r in results])
    
    # Distribution of predictions
    pred_counts = [r['num_predictions'] for r in results]
    gt_counts = [r['num_ground_truth'] for r in results]
    
    print("\n" + "="*60)
    print("📊 SUMMARY STATISTICS")
    print("="*60)
    
    print(f"\n🎯 Hit Rate:")
    print(f"   Sentences with ≥1 correct prediction: {sentences_with_match}/{len(results)} ({100*sentences_with_match/len(results):.1f}%)")
    
    print(f"\n📈 Average Per-Sentence Metrics:")
    print(f"   Avg Precision: {avg_precision:.3f}")
    print(f"   Avg Recall:    {avg_recall:.3f}")
    print(f"   Avg F1:        {avg_f1:.3f}")
    
    print(f"\n📋 Prediction Distribution:")
    print(f"   Avg predictions per sentence: {np.mean(pred_counts):.1f}")
    print(f"   Avg ground truth per sentence: {np.mean(gt_counts):.1f}")
    
    print(f"\n🔍 Coverage:")
    print(f"   Sentences with 0 predictions: {sum(1 for p in pred_counts if p == 0)}")
    print(f"   Sentences with 5+ predictions: {sum(1 for p in pred_counts if p >= 5)}")

print_summary_statistics(eval_results)


📊 SUMMARY STATISTICS

🎯 Hit Rate:
   Sentences with ≥1 correct prediction: 3/200 (1.5%)

📈 Average Per-Sentence Metrics:
   Avg Precision: 0.002
   Avg Recall:    0.013
   Avg F1:        0.003

📋 Prediction Distribution:
   Avg predictions per sentence: 10.0
   Avg ground truth per sentence: 1.4

🔍 Coverage:
   Sentences with 0 predictions: 0
   Sentences with 5+ predictions: 200
